In [1]:
from google.cloud import storage
import os
from io import StringIO # if going with no saving csv file
import gcsfs
import dask.dataframe as dd
from dask.delayed import delayed
from pathlib import Path
import pandas as pd
import os

In [2]:
from pprint import pprint, pp
from IPython.display import display, Markdown
from icecream import ic

In [3]:
data_dir = Path(os.getenv('APP_DIR') + '/data')

userlogin_datadir = Path(data_dir, 'nanohub_metrics/userlogin')
jos_users_datadir = Path(data_dir, 'nanohub/jos_users')

In [4]:
filenames = userlogin_datadir.glob('*.parquet.gzip')
dfs = [delayed(pd.read_parquet)(fn) for fn in filenames]

dask_df = dd.from_delayed(dfs) 
dask_df = dask_df.persist()
userlogin_df = dask_df.compute(scheduler='processes')


In [5]:
display(userlogin_df.head())
display(userlogin_df.tail())

,datetime,user,uidnumber,ip,action
id,,,,,
264497646,2018-02-19 04:39:48,-,0,157.55.39.123,detect
264497647,2018-02-19 04:39:49,-,0,14.139.181.229,detect
264497648,2018-02-19 04:39:49,-,0,46.229.168.80,detect
264497649,2018-02-19 04:39:50,-,0,118.185.164.5,detect
264497650,2018-02-19 04:39:51,-,0,14.139.181.229,detect


,datetime,user,uidnumber,ip,action
id,,,,,
108497641,2015-08-03 02:14:03,-,0,68.180.229.165,detect
108497642,2015-08-03 02:14:04,-,0,66.249.69.71,detect
108497643,2015-08-03 02:14:04,-,0,68.180.229.165,detect
108497644,2015-08-03 02:14:06,-,0,74.15.149.43,detect
108497645,2015-08-03 02:14:06,-,0,61.16.171.242,detect


In [6]:
jos_users_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in jos_users_datadir.glob('*.parquet.gzip')
)

In [7]:
display(jos_users_df.head())
display(jos_users_df.tail())

,name,username,email,lastvisitDate,registerDate
id,,,,,
998,hubrepo hubrepo,hubrepo,nkissebe@gmail.com,NaT,2014-11-13 21:09:09
1000,CMS Admin Manager,admin,support@nanohub.org,NaT,NaT
1683,nanoHUB support,support,support@nanohub.org,2008-11-19 23:55:30,2008-11-19 22:51:04
1684,Grid Statistics,gridstat,gridstat@nanohub.org,2020-02-14 18:50:14,2008-11-18 17:29:56
1685,NCN NCN,ncn,ncn@nanohub.org,NaT,2008-11-11 19:17:04


,name,username,email,lastvisitDate,registerDate
id,,,,,
332875,Katherine Caroline,katherinecarolynd9890,katherinecarolynd9890@gmail.com,2021-07-17 09:11:38,2021-07-17 09:11:38
332876,buy IG followers from Insta4likes,insta4likes,insta4likes@protonmail.com,2021-07-17 09:42:08,2021-07-17 09:41:15
332879,Richard Boice,rboice,ricky.boice@gmail.com,2021-07-17 09:45:45,2021-07-17 09:45:45
332880,joe nick,knighten,knighten@gmail.com,2021-07-17 09:56:08,2021-07-17 09:56:08
332881,Kenneth Parker,-174570,-174570@invalid,2021-07-17 10:18:56,2021-07-17 10:18:55


In [8]:
display(userlogin_df.dtypes)

datetime     datetime64[ns]
user                 object
uidnumber             int64
ip                   object
action               object
dtype: object

In [ ]:
action_types_df = userlogin_df.groupby('action')['user'].unique().reset_index()
pd.options.display.max_colwidth = 5000
display(action_types_df)
pd.reset_option('max_colwidth')

In [ ]:
display(userlogin_df.iloc[0])

In [ ]:
ddf_by_datetime = dask_df.set_index('datetime')
display(ddf_by_datetime.head())

In [ ]:
def get_users_df_between_yrs(df: pd.DataFrame, yr1: int, yr2: int):
    return df[(df['datetime'] > str(yr1)) & (df['datetime'] < str(yr2))]

def yearly_count_users(df: pd.DataFrame, start_yr: int, end_yr: int):
    count = {}
    while start_yr <= end_yr:
        count[start_yr] = get_users_df_between_yrs(df, start_yr, start_yr + 1).shape[0]
        start_yr = start_yr + 1
        
    return count

count_users = yearly_count_users(ddf_by_datetime.compute(), 2000, 2021)
count_users_df = pd.DataFrame.from_dict(count_users, orient='index', columns=['user count'])
display(count_users_df)

In [ ]:
def get_users_df_between_yrs(df, yr1: int, yr2: int):
    return df[df['datetime'] > str(yr1)]

def yearly_count_users(df, start_yr: int, end_yr: int):
    count = {}
    while start_yr <= end_yr:
        count[start_yr] = get_users_df_between_yrs(df, start_yr, start_yr + 1).shape[0]
        start_yr = start_yr + 1
        
    return count

count_users = yearly_count_users(ddf_by_datetime, 2000, 2021)
count_users_df = pd.DataFrame.from_dict(count_users, orient='index', columns=['user count'])
display(count_users_df)